In [1]:
%matplotlib inline


Doc2Vec Model
=============

Introduces Gensim's Doc2Vec model and demonstrates its use on the Lee Corpus.




In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Doc2Vec is a `core_concepts_model` that represents each
`core_concepts_document` as a `core_concepts_vector`.  This
tutorial introduces the model and demonstrates how to train and assess it.

Here's a list of what we'll be doing:

0. Review the relevant models: bag-of-words, Word2Vec, Doc2Vec
1. Load and preprocess the training and test corpora (see `core_concepts_corpus`)
2. Train a Doc2Vec `core_concepts_model` model using the training corpus
3. Demonstrate how the trained model can be used to infer a `core_concepts_vector`
4. Assess the model
5. Test the model on the test corpus

Review: Bag-of-words
--------------------

.. Note:: Feel free to skip these review sections if you're already familiar with the models.

You may be familiar with the `bag-of-words model
<https://en.wikipedia.org/wiki/Bag-of-words_model>`_ from the
`core_concepts_vector` section.
This model transforms each document to a fixed-length vector of integers.
For example, given the sentences:

- ``John likes to watch movies. Mary likes movies too.``
- ``John also likes to watch football games. Mary hates football.``

The model outputs the vectors:

- ``[1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]``
- ``[1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]``

Each vector has 10 elements, where each element counts the number of times a
particular word occurred in the document.
The order of elements is arbitrary.
In the example above, the order of the elements corresponds to the words:
``["John", "likes", "to", "watch", "movies", "Mary", "too", "also", "football", "games", "hates"]``.

Bag-of-words models are surprisingly effective, but have several weaknesses.

First, they lose all information about word order: "John likes Mary" and
"Mary likes John" correspond to identical vectors. There is a solution: bag
of `n-grams <https://en.wikipedia.org/wiki/N-gram>`__
models consider word phrases of length n to represent documents as
fixed-length vectors to capture local word order but suffer from data
sparsity and high dimensionality.

Second, the model does not attempt to learn the meaning of the underlying
words, and as a consequence, the distance between vectors doesn't always
reflect the difference in meaning.  The ``Word2Vec`` model addresses this
second problem.

Review: ``Word2Vec`` Model
--------------------------

``Word2Vec`` is a more recent model that embeds words in a lower-dimensional
vector space using a shallow neural network. The result is a set of
word-vectors where vectors close together in vector space have similar
meanings based on context, and word-vectors distant to each other have
differing meanings. For example, ``strong`` and ``powerful`` would be close
together and ``strong`` and ``Paris`` would be relatively far.

Gensim's :py:class:`~gensim.models.word2vec.Word2Vec` class implements this model.

With the ``Word2Vec`` model, we can calculate the vectors for each **word** in a document.
But what if we want to calculate a vector for the **entire document**\ ?
We could average the vectors for each word in the document - while this is quick and crude, it can often be useful.
However, there is a better way...

Introducing: Paragraph Vector
-----------------------------

.. Important:: In Gensim, we refer to the Paragraph Vector model as ``Doc2Vec``.

Le and Mikolov in 2014 introduced the `Doc2Vec algorithm <https://cs.stanford.edu/~quocle/paragraph_vector.pdf>`__, which usually outperforms such simple-averaging of ``Word2Vec`` vectors.

The basic idea is: act as if a document has another floating word-like
vector, which contributes to all training predictions, and is updated like
other word-vectors, but we will call it a doc-vector. Gensim's
:py:class:`~gensim.models.doc2vec.Doc2Vec` class implements this algorithm.

There are two implementations:

1. Paragraph Vector - Distributed Memory (PV-DM)
2. Paragraph Vector - Distributed Bag of Words (PV-DBOW)

.. Important::
  Don't let the implementation details below scare you.
  They're advanced material: if it's too much, then move on to the next section.

PV-DM is analogous to Word2Vec CBOW. The doc-vectors are obtained by training
a neural network on the synthetic task of predicting a center word based an
average of both context word-vectors and the full document's doc-vector.

PV-DBOW is analogous to Word2Vec SG. The doc-vectors are obtained by training
a neural network on the synthetic task of predicting a target word just from
the full document's doc-vector. (It is also common to combine this with
skip-gram testing, using both the doc-vector and nearby word-vectors to
predict a single target word, but only one at a time.)

Prepare the Training and Test Data
----------------------------------

For this tutorial, we'll be training our model using the `Lee Background
Corpus
<https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf>`_
included in gensim. This corpus contains 314 documents selected from the
Australian Broadcasting Corporation’s news mail service, which provides text
e-mails of headline stories and covers a number of broad topics.

And we'll test our model by eye using the much shorter `Lee Corpus
<https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf>`_
which contains 50 documents.




In [2]:
import os
import gensim
# Set file names for train and test data
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

Define a Function to Read and Preprocess Text
---------------------------------------------

Below, we define a function to:

- open the train/test file (with latin encoding)
- read the file line-by-line
- pre-process each line (tokenize text into individual words, remove punctuation, set to lowercase, etc)

The file we're reading is a **corpus**.
Each line of the file is a **document**.

.. Important::
  To train the model, we'll need to associate a tag/number with each document
  of the training corpus. In our case, the tag is simply the zero-based line
  number.




In [3]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

Let's take a look at the training corpus




In [4]:
print(train_corpus[:2])

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

And the testing corpus looks like this:




In [5]:
print(test_corpus[:2])

[['the', 'national', 'executive', 'of', 'the', 'strife', 'torn', 'democrats', 'last', 'night', 'appointed', 'little', 'known', 'west', 'australian', 'senator', 'brian', 'greig', 'as', 'interim', 'leader', 'shock', 'move', 'likely', 'to', 'provoke', 'further', 'conflict', 'between', 'the', 'party', 'senators', 'and', 'its', 'organisation', 'in', 'move', 'to', 'reassert', 'control', 'over', 'the', 'party', 'seven', 'senators', 'the', 'national', 'executive', 'last', 'night', 'rejected', 'aden', 'ridgeway', 'bid', 'to', 'become', 'interim', 'leader', 'in', 'favour', 'of', 'senator', 'greig', 'supporter', 'of', 'deposed', 'leader', 'natasha', 'stott', 'despoja', 'and', 'an', 'outspoken', 'gay', 'rights', 'activist'], ['cash', 'strapped', 'financial', 'services', 'group', 'amp', 'has', 'shelved', 'million', 'plan', 'to', 'buy', 'shares', 'back', 'from', 'investors', 'and', 'will', 'raise', 'million', 'in', 'fresh', 'capital', 'after', 'profits', 'crashed', 'in', 'the', 'six', 'months', 'to'

Notice that the testing corpus is just a list of lists and does not contain
any tags.




Training the Model
------------------

Now, we'll instantiate a Doc2Vec model with a vector size with 50 dimensions and
iterating over the training corpus 40 times. We set the minimum word count to
2 in order to discard words with very few occurrences. (Without a variety of
representative examples, retaining such infrequent words can often make a
model worse!) Typical iteration counts in the published `Paragraph Vector paper <https://cs.stanford.edu/~quocle/paragraph_vector.pdf>`__
results, using 10s-of-thousands to millions of docs, are 10-20. More
iterations take more time and eventually reach a point of diminishing
returns.

However, this is a very very small dataset (300 documents) with shortish
documents (a few hundred words). Adding training passes can sometimes help
with such small datasets.




In [8]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

2020-08-19 15:34:52,182 : WARNING : consider setting layer size to a multiple of 4 for greater performance


Build a vocabulary



In [9]:
model.build_vocab(train_corpus)

2020-08-19 15:34:57,144 : INFO : collecting all words and their counts
2020-08-19 15:34:57,145 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-08-19 15:34:57,153 : INFO : collected 6981 word types and 300 unique tags from a corpus of 300 examples and 58152 words
2020-08-19 15:34:57,154 : INFO : Loading a fresh vocabulary
2020-08-19 15:34:57,162 : INFO : effective_min_count=2 retains 3955 unique words (56% of original 6981, drops 3026)
2020-08-19 15:34:57,163 : INFO : effective_min_count=2 leaves 55126 word corpus (94% of original 58152, drops 3026)
2020-08-19 15:34:57,174 : INFO : deleting the raw counts dictionary of 6981 items
2020-08-19 15:34:57,175 : INFO : sample=0.001 downsamples 46 most-common words
2020-08-19 15:34:57,176 : INFO : downsampling leaves estimated 42390 word corpus (76.9% of prior 55126)
2020-08-19 15:34:57,184 : INFO : estimated required memory for 3955 words and 50 dimensions: 3619500 bytes
2020-08-19 15:34:57,185 : INFO : res

Essentially, the vocabulary is a dictionary (accessible via
``model.wv.vocab``\ ) of all of the unique words extracted from the training
corpus along with the count (e.g., ``model.wv.vocab['penalty'].count`` for
counts for the word ``penalty``\ ).




Next, train the model on the corpus.
If the BLAS library is being used, this should take no more than 3 seconds.
If the BLAS library is not being used, this should take no more than 2
minutes, so use BLAS if you value your time.




In [10]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2020-08-19 15:35:20,437 : INFO : training model with 3 workers on 3955 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-08-19 15:35:20,482 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-19 15:35:20,484 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-19 15:35:20,485 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-19 15:35:20,486 : INFO : EPOCH - 1 : training on 58152 raw words (42626 effective words) took 0.0s, 907459 effective words/s
2020-08-19 15:35:20,530 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-19 15:35:20,532 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-19 15:35:20,534 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-19 15:35:20,534 : INFO : EPOCH - 2 : training on 58152 raw words (42717 effective words) took 0.0s, 929453 effective words/s
2020-08-19 15:35:20,570 : INFO : worker 

2020-08-19 15:35:21,373 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-19 15:35:21,375 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-19 15:35:21,377 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-19 15:35:21,377 : INFO : EPOCH - 21 : training on 58152 raw words (42703 effective words) took 0.0s, 1036144 effective words/s
2020-08-19 15:35:21,418 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-19 15:35:21,419 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-19 15:35:21,420 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-19 15:35:21,420 : INFO : EPOCH - 22 : training on 58152 raw words (42825 effective words) took 0.0s, 1088625 effective words/s
2020-08-19 15:35:21,462 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-19 15:35:21,463 : INFO : worker thread finished; awaiting finish of 1 more threads
20

Now, we can use the trained model to infer a vector for any piece of text
by passing a list of words to the ``model.infer_vector`` function. This
vector can then be compared with other vectors via cosine similarity.




In [16]:
vector1 = model.infer_vector(['Soccer', 'is', 'a', 'popular', 'sport', 'always'])
print(vector1)

[-0.01490097  0.02111813 -0.03109653 -0.01068063 -0.01892539 -0.3459541
  0.08585836  0.05513358  0.23179501 -0.03962927 -0.04393085 -0.26241705
  0.1303937  -0.07906086 -0.19340687 -0.08851517 -0.12310801 -0.14102398
  0.0950866  -0.03843781 -0.01226297  0.13106963  0.04221071  0.14866176
 -0.11223336 -0.06549908  0.28320494  0.11926895 -0.27281046 -0.12527478
  0.0416162   0.25637335 -0.10573249  0.10783476  0.335377    0.0543611
 -0.20440938  0.14640705  0.01357154 -0.08848789  0.0082607   0.15403083
  0.01400404  0.05685166  0.04151241  0.05091893  0.04578988 -0.04650984
 -0.09706578  0.30933914]


In [29]:
vector2 = model.infer_vector(['Football', 'is', 'very', 'fun', 'to', 'play'])
print(vector2)

[-0.09620431  0.08249308 -0.0395282  -0.01915498  0.07021865 -0.18614933
  0.16727541  0.07942826  0.18905391 -0.18667327 -0.03328803 -0.10956103
  0.11144634 -0.15424423 -0.10160239 -0.0665291  -0.23383522  0.01524642
 -0.03415006 -0.0788225   0.00722739  0.10200074  0.15445967  0.03229436
 -0.14097126  0.00884841  0.27664086  0.0468643  -0.16139928 -0.0074278
 -0.04843001  0.25789395 -0.15133135  0.1747285   0.19457853  0.114756
 -0.2919086   0.02184635  0.0183157  -0.129457    0.08781081  0.01398888
 -0.07676763  0.03834012  0.03356643  0.0736498   0.1731747  -0.01246149
 -0.08871602  0.23394802]


In [30]:
res = vector1 - vector2
import numpy as np
import math
res = sum(abs(res))
print(res)

3.468309050425887


Note that ``infer_vector()`` does *not* take a string, but rather a list of
string tokens, which should have already been tokenized the same way as the
``words`` property of original training document objects.

Also note that because the underlying training/inference algorithms are an
iterative approximation problem that makes use of internal randomization,
repeated inferences of the same text will return slightly different vectors.




Assessing the Model
-------------------

To assess our new model, we'll first infer new vectors for each document of
the training corpus, compare the inferred vectors with the training corpus,
and then returning the rank of the document based on self-similarity.
Basically, we're pretending as if the training corpus is some new unseen data
and then seeing how they compare with the trained model. The expectation is
that we've likely overfit our model (i.e., all of the ranks will be less than
2) and so we should be able to find similar documents very easily.
Additionally, we'll keep track of the second ranks for a comparison of less
similar documents.




In [31]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

2020-08-19 15:42:55,033 : INFO : precomputing L2-norms of doc weight vectors


Let's count how each document ranks with respect to the training corpus

NB. Results vary between runs due to random seeding and very small corpus



In [32]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 293, 1: 7})


Basically, greater than 95% of the inferred documents are found to be most
similar to itself and about 5% of the time it is mistakenly most similar to
another document. Checking the inferred-vector against a
training-vector is a sort of 'sanity check' as to whether the model is
behaving in a usefully consistent manner, though not a real 'accuracy' value.

This is great and not entirely surprising. We can take a look at an example:




In [33]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Notice above that the most similar document (usually the same text) is has a
similarity score approaching 1.0. However, the similarity score for the
second-ranked documents should be significantly lower (assuming the documents
are in fact different) and the reasoning becomes obvious when we examine the
text itself.

We can run the next cell repeatedly to see a sampling other target-document
comparisons.




In [34]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (62): «japanese officials say their coast guard has sunk an unidentified boat after an exchange of fire in the east china sea the bbc reports the unidentified boat which resembled fishing trawler was spotted cruising off south western japan by naval reconnaissance plane more than japanese coastguard vessels were mobilised to give chase japanese officials said warning shots were fired on several occasions but the boat ignored orders to stop and continued heading west towards china the officials said crew members appeared on deck brandishing metal pipes and several hours after the chase began there was an exchange of gunfire in which two japanese sailors were injured the patrol boats then sank the vessel and its man crew was thrown into the sea some reports said it resembled north korean spy boat but there was also speculation it could have held chinese smugglers meanwhile the coast guard is continuing efforts to recover crew members six of the on board have so far been re

Testing the Model
-----------------

Using the same approach above, we'll infer the vector for a randomly chosen
test document, and compare the document to our model by eye.




In [41]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
doc_id = ["Which", "soccer", "player", "was", "injured"]
#inferred_vector = model.infer_vector(test_corpus[doc_id])
inferred_vector = model.infer_vector(doc_id)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
#print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(doc_id)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (['Which', 'soccer', 'player', 'was', 'injured']): «Which soccer player was injured»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (96, 0.6286325454711914): «england batsman michael vaughan has become just the seventh player in the history of test cricket to be given out handling the ball vaughan was on when he flicked the ball with his hand towards fielder on the first day of the third and final test against india at bangalore the englishmen says he was disappointed in the appeal by the indians which he believes was not in the spirit of the game in the laws shouldn have done it but thought was just helping out the fielders he said feel bit disappointed on their behalf one of their players appealed not too sure who it was but bit disappointed really england finished the opening day at for mark ramprakash was out for he and vaughan adding for the fourth wicket after england was struggling at for staff at the woomera detention centre are st

Conclusion
----------

Let's review what we've seen in this tutorial:

0. Review the relevant models: bag-of-words, Word2Vec, Doc2Vec
1. Load and preprocess the training and test corpora (see `core_concepts_corpus`)
2. Train a Doc2Vec `core_concepts_model` model using the training corpus
3. Demonstrate how the trained model can be used to infer a `core_concepts_vector`
4. Assess the model
5. Test the model on the test corpus

That's it! Doc2Vec is a great way to explore relationships between documents.

Additional Resources
--------------------

If you'd like to know more about the subject matter of this tutorial, check out the links below.

* `Word2Vec Paper <https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf>`_
* `Doc2Vec Paper <https://cs.stanford.edu/~quocle/paragraph_vector.pdf>`_
* `Dr. Michael D. Lee's Website <http://faculty.sites.uci.edu/mdlee>`_
* `Lee Corpus <http://faculty.sites.uci.edu/mdlee/similarity-data/>`__
* `IMDB Doc2Vec Tutorial <doc2vec-IMDB.ipynb>`_


